## Import Libraries

In [1]:
# standard library imports
import csv
import datetime as dt
import json
import os
import statistics
import time

# third-party imports
import numpy as np
import pandas as pd
import requests

# customisations - ensure tables show all columns
pd.set_option("display.max_columns", 100)

In [2]:
def get_request(url, parameters=None):
    """Return json-formatted response of a get request using optional parameters.
    
    Parameters
    ----------
    url : string
    parameters : {'parameter': 'value'}
        parameters to pass as part of get request
    
    Returns
    -------
    json_data
        json-formatted response (dict-like)
    """
    try:
        response = requests.get(url=url, params=parameters)
    except SSLError as s:
        print('SSL Error:', s)
        
        for i in range(5, 0, -1):
            print('\rWaiting... ({})'.format(i), end='')
            time.sleep(1)
        print('\rRetrying.' + ' '*10)
        
        # recusively try again
        return get_request(url, parameters)
    
    if response:
        return response.json()
    else:
        # response is none usually means too many requests. Wait and try again 
        print('No response, waiting 10 seconds...')
        time.sleep(10)
        print('Retrying.')
        return get_request(url, parameters)

## Generate List of App IDs

In [3]:
url = "https://steamspy.com/api.php"
parameters = {"request": "all"}

# request 'all' from steam spy and parse into dataframe
json_data = get_request(url, parameters=parameters)
steam_spy_all = pd.DataFrame.from_dict(json_data, orient='index')

# generate sorted app_list from steamspy data
app_list = steam_spy_all[['appid', 'name']].sort_values('appid').reset_index(drop=True)

# export disabled to keep consistency across download sessions
# app_list.to_csv('../Data/Download/app_list.csv', index=False)

# instead read from stored csv
app_list = pd.read_csv('../Data/Download/app_list.csv')

# display first few rows
app_list.head()

,appid,name
0,40,Deathmatch Classic
1,1200,Red Orchestra: Ostfront 41-45
2,1510,Uplink
3,1530,Multiwinia
4,1900,Earth 2160


## Define Download Logic

In [4]:
def get_app_data(start, stop, parser, pause):
    """Return list of app data generated from parser.
    
    parser : function to handle request
    """
    app_data = []
    
    # iterate through each row of app_list, confined by start and stop
    for index, row in app_list[start:stop].iterrows():
        print('Current index: {}'.format(index), end='\r')
        
        appid = row['appid']
        name = row['name']

        # retrive app data for a row, handled by supplied parser, and append to list
        data = parser(appid, name)
        app_data.append(data)

        time.sleep(pause) # prevent overloading api with requests
    
    return app_data


def process_batches(parser, app_list, download_path, data_filename, index_filename,
                    columns, begin=0, end=-1, batchsize=100, pause=1):
    """Process app data in batches, writing directly to file.
    
    parser : custom function to format request
    app_list : dataframe of appid and name
    download_path : path to store data
    data_filename : filename to save app data
    index_filename : filename to store highest index written
    columns : column names for file
    
    Keyword arguments:
    
    begin : starting index (get from index_filename, default 0)
    end : index to finish (defaults to end of app_list)
    batchsize : number of apps to write in each batch (default 100)
    pause : time to wait after each api request (defualt 1)
    
    returns: none
    """
    print('Starting at index {}:\n'.format(begin))
    
    # by default, process all apps in app_list
    if end == -1:
        end = len(app_list) + 1
    
    # generate array of batch begin and end points
    batches = np.arange(begin, end, batchsize)
    batches = np.append(batches, end)
    
    apps_written = 0
    batch_times = []
    
    for i in range(len(batches) - 1):
        start_time = time.time()
        
        start = batches[i]
        stop = batches[i+1]
        
        app_data = get_app_data(start, stop, parser, pause)
        
        rel_path = os.path.join(download_path, data_filename)
        
        # writing app data to file
        with open(rel_path, 'a', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=columns, extrasaction='ignore')
            
            for j in range(3,0,-1):
                print("\rAbout to write data, don't stop script! ({})".format(j), end='')
                time.sleep(0.5)
            
            writer.writerows(app_data)
            print('\rExported lines {}-{} to {}.'.format(start, stop-1, data_filename), end=' ')
            
        apps_written += len(app_data)
        
        idx_path = os.path.join(download_path, index_filename)
        
        # writing last index to file
        with open(idx_path, 'w') as f:
            index = stop
            print(index, file=f)
            
        # logging time taken
        end_time = time.time()
        time_taken = end_time - start_time
        
        batch_times.append(time_taken)
        mean_time = statistics.mean(batch_times)
        
        est_remaining = (len(batches) - i - 2) * mean_time
        
        remaining_td = dt.timedelta(seconds=round(est_remaining))
        time_td = dt.timedelta(seconds=round(time_taken))
        mean_td = dt.timedelta(seconds=round(mean_time))
        
        print('Batch {} time: {} (avg: {}, remaining: {})'.format(i, time_td, mean_td, remaining_td))
            
    print('\nProcessing batches complete. {} apps written'.format(apps_written))

In [5]:
def reset_index(download_path, index_filename):
    """Reset index in file to 0."""
    rel_path = os.path.join(download_path, index_filename)
    
    with open(rel_path, 'w') as f:
        print(0, file=f)
        

def get_index(download_path, index_filename):
    """Retrieve index from file, returning 0 if file not found."""
    try:
        rel_path = os.path.join(download_path, index_filename)

        with open(rel_path, 'r') as f:
            index = int(f.readline())
    
    except FileNotFoundError:
        index = 0
        
    return index


def prepare_data_file(download_path, filename, index, columns):
    """Create file and write headers if index is 0."""
    if index == 0:
        rel_path = os.path.join(download_path, filename)

        with open(rel_path, 'w', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=columns)
            writer.writeheader()

## Download Steam Data

In [6]:
def parse_steam_request(appid, name):
    """Unique parser to handle data from Steam Store API.
    
    Returns : json formatted data (dict-like)
    """
    url = "http://store.steampowered.com/api/appdetails/"
    parameters = {"appids": appid}
    
    json_data = get_request(url, parameters=parameters)
    json_app_data = json_data[str(appid)]
    
    if json_app_data['success']:
        data = json_app_data['data']
    else:
        data = {'name': name, 'steam_appid': appid}
        
    return data


# Set file parameters
download_path = '../Data/Download'
steam_app_data = 'steam_app_data.csv'
steam_index = 'steam_index.txt'

steam_columns = [
    'type', 'name', 'steam_appid', 'required_age', 'is_free', 'controller_support',
    'dlc', 'detailed_description', 'about_the_game', 'short_description', 'fullgame',
    'supported_languages', 'header_image', 'website', 'pc_requirements', 'mac_requirements',
    'linux_requirements', 'legal_notice', 'drm_notice', 'ext_user_account_notice',
    'developers', 'publishers', 'demos', 'price_overview', 'packages', 'package_groups',
    'platforms', 'metacritic', 'reviews', 'categories', 'genres', 'screenshots',
    'movies', 'recommendations', 'achievements', 'release_date', 'support_info',
    'background', 'content_descriptors'
]

# Overwrites last index for demonstration (would usually store highest index so can continue across sessions)
reset_index(download_path, steam_index)

# Retrieve last index downloaded from file
index = get_index(download_path, steam_index)

# Wipe or create data file and write headers if index is 0
prepare_data_file(download_path, steam_app_data, index, steam_columns)

# Set end and chunksize for demonstration - remove to run through entire app list
process_batches(
    parser=parse_steam_request,
    app_list=app_list,
    download_path=download_path,
    data_filename=steam_app_data,
    index_filename=steam_index,
    columns=steam_columns,
    begin=index,
    end=10,
    batchsize=5
)

Starting at index 0:

Exported lines 0-4 to steam_app_data.csv. Batch 0 time: 0:00:09 (avg: 0:00:09, remaining: 0:00:09)
Exported lines 5-9 to steam_app_data.csv. Batch 1 time: 0:00:09 (avg: 0:00:09, remaining: 0:00:00)

Processing batches complete. 10 apps written


In [7]:
# inspect downloaded data
pd.read_csv('../Data/Download/steam_app_data.csv').head()

,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,fullgame,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors
0,game,Deathmatch Classic,40,0,False,NaN,NaN,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,NaN,"English, French, German, Italian, Spanish - Sp...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'SGD', 'initial': 525, 'final': 5...",[31],"[{'name': 'default', 'title': 'Buy Deathmatch ...","{'windows': True, 'mac': True, 'linux': True}",NaN,NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 1774},NaN,"{'coming_soon': False, 'date': '1 Jun, 2001'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
1,game,Red Orchestra: Ostfront 41-45,1200,0,False,NaN,NaN,<h1>D-Day Anniversary 2016 Update for Darkest ...,Fight in the theatre of war that changed the w...,Fight in the theatre of war that changed the w...,NaN,"English, French, Russian",https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.redorchestragame.com/,{'minimum': '<strong>Minimum:</strong><br><ul ...,"{'minimum': '<ul class=""bb_ul""><li><strong>OS:...",{'minimum': '<strong>Minimum:</strong><br><ul ...,NaN,NaN,NaN,['Tripwire Interactive'],['Tripwire Interactive'],NaN,"{'currency': 'SGD', 'initial': 525, 'final': 2...",[63],"[{'name': 'default', 'title': 'Buy Red Orchest...","{'windows': True, 'mac': True, 'linux': True}","{'score': 81, 'url': 'https://www.metacritic.c...",<strong>&quot;... RO is also one of the market...,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 1303},"{'total': 44, 'highlighted': [{'name': 'Kills ...","{'coming_soon': False, 'date': '14 Mar, 2006'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
2,game,Uplink,1510,0,False,NaN,NaN,You play an Uplink Agent who makes a living by...,You play an Uplink Agent who makes a living by...,You play an Uplink Agent who makes a living by...,NaN,English,https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.uplink.co.uk/,[],[],[],NaN,NaN,NaN,['Introversion Software'],['Introversion Software'],NaN,"{'currency': 'SGD', 'initial': 1000, 'final': ...","[112, 14002]","[{'name': 'default', 'title': 'Buy Uplink', 'd...","{'windows': True, 'mac': True, 'linux': True}","{'score': 75, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 2, 'description': 'Single-player'}]","[{'id': '23', 'description': 'Indie'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 1578},NaN,"{'coming_soon': False, 'date': '23 Aug, 2006'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
3,game,Multiwinia,1530,0,False,NaN,NaN,&quot;Long ago a computer scientist called Dr ...,&quot;Long ago a computer scientist called Dr ...,&quot;Long ago a computer scientist called Dr ...,NaN,"English, French, Italian, German, Spanish - Spain",https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.introversion.co.uk/multiwinia/,"{'minimum': '<ul class=""bb_ul""><li><strong>Sup...","{'minimum': '<ul class=""bb_ul""><li><strong>OS:...","{'minimum'

## Download SteamSpy Data

In [8]:
def parse_steamspy_request(appid, name):
    """Parser to handle SteamSpy API data."""
    url = "https://steamspy.com/api.php"
    parameters = {"request": "appdetails", "appid": appid}
    
    json_data = get_request(url, parameters)
    return json_data


# set files and columns
download_path = '../Data/Download'
steamspy_data = 'steamspy_data.csv'
steamspy_index = 'steamspy_index.txt'

steamspy_columns = [
    'appid', 'name', 'developer', 'publisher', 'score_rank', 'positive',
    'negative', 'userscore', 'owners', 'average_forever', 'average_2weeks',
    'median_forever', 'median_2weeks', 'price', 'initialprice', 'discount',
    'languages', 'genre', 'ccu', 'tags'
]

reset_index(download_path, steamspy_index)
index = get_index(download_path, steamspy_index)

# Wipe data file if index is 0
prepare_data_file(download_path, steamspy_data, index, steamspy_columns)

process_batches(
    parser=parse_steamspy_request,
    app_list=app_list,
    download_path=download_path, 
    data_filename=steamspy_data,
    index_filename=steamspy_index,
    columns=steamspy_columns,
    begin=index,
    end=20,
    batchsize=5,
    pause=0.3
)

Starting at index 0:

Exported lines 0-4 to steamspy_data.csv. Batch 0 time: 0:00:06 (avg: 0:00:06, remaining: 0:00:19)
Exported lines 5-9 to steamspy_data.csv. Batch 1 time: 0:00:06 (avg: 0:00:06, remaining: 0:00:12)
Exported lines 10-14 to steamspy_data.csv. Batch 2 time: 0:00:10 (avg: 0:00:07, remaining: 0:00:07)
Exported lines 15-19 to steamspy_data.csv. Batch 3 time: 0:00:07 (avg: 0:00:07, remaining: 0:00:00)

Processing batches complete. 20 apps written


In [9]:
# inspect downloaded steamspy data
pd.read_csv('../Data/Download/steamspy_data.csv').head()

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
0,40,Deathmatch Classic,Valve,Valve,NaN,2098,467,0,"5,000,000 .. 10,000,000",17473,19212,13,19212,499,499,0,"English, French, German, Italian, Spanish - Sp...",Action,3,"{'Action': 632, 'FPS': 145, 'Classic': 111, 'M..."
1,1200,Red Orchestra: Ostfront 41-45,Tripwire Interactive,Tripwire Interactive,NaN,1989,302,0,"500,000 .. 1,000,000",391,0,60,0,249,499,50,"English, French, Russian",Action,33,"{'World War II': 156, 'Action': 126, 'FPS': 12..."
2,1510,Uplink,Introversion Software,Introversion Software,NaN,2014,193,0,"500,000 .. 1,000,000",298,0,75,0,999,999,0,English,"Indie, Strategy",9,"{'Hacking': 226, 'Indie': 160, 'Strategy': 146..."
3,1530,Multiwinia,Introversion Software,Introversion Software,NaN,444,99,0,"500,000 .. 1,000,000",45,0,68,0,999,999,0,"English, French, Italian, German, Spanish - Spain",Indie,2,"{'Indie': 57, 'RTS': 35, 'Strategy': 33, 'Mult..."
4,1900,Earth 2160,Reality Pump Studios,"Topware Interactive, ACE",NaN,878,374,0,"500,000 .. 1,000,000",1015,0,423,0,149,499,70,"English, French, German, Spanish - Spain, Ital...",Strategy,11,"{'Base-Building': 960, 'Space': 954, 'RTS': 95..."
